In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum((1 for char in text if char in string.punctuation))
    return round(count/(len(text) - text.count(' ')), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis = 1) 
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'n_features', 'predict', 'predict_log_proba', 'predict_proba', 'score', 's

In [4]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size= 0.2)

In [15]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label= "spam", average = 'binary')
    print('Est: {} / Depth: {} / LR: {} ------- precision: {} / recall: {} / Acurracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [ ]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

C:\Users\dynamic_h\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ------- precision: 0.0 / recall: 0.0 / Acurracy: 0.868
Est: 50 / Depth: 3 / LR: 0.1 ------- precision: 0.959 / recall: 0.796 / Acurracy: 0.969
Est: 50 / Depth: 3 / LR: 1 ------- precision: 0.923 / recall: 0.816 / Acurracy: 0.967
Est: 50 / Depth: 7 / LR: 0.01 ------- precision: 1.0 / recall: 0.014 / Acurracy: 0.87
Est: 50 / Depth: 7 / LR: 0.1 ------- precision: 0.947 / recall: 0.857 / Acurracy: 0.975
Est: 50 / Depth: 7 / LR: 1 ------- precision: 0.919 / recall: 0.85 / Acurracy: 0.97
Est: 50 / Depth: 11 / LR: 0.01 ------- precision: 1.0 / recall: 0.034 / Acurracy: 0.873
Est: 50 / Depth: 11 / LR: 0.1 ------- precision: 0.949 / recall: 0.884 / Acurracy: 0.978
Est: 50 / Depth: 11 / LR: 1 ------- precision: 0.931 / recall: 0.912 / Acurracy: 0.979
Est: 50 / Depth: 15 / LR: 0.01 ------- precision: 1.0 / recall: 0.014 / Acurracy: 0.87
Est: 50 / Depth: 15 / LR: 0.1 ------- precision: 0.936 / recall: 0.891 / Acurracy: 0.978
Est: 50 / Depth: 15 / LR: 1 ------- precisi

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum((1 for char in text if char in string.punctuation))
    return round(count/(len(text) - text.count(' ')), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis = 1)

# CountVectorizer
count_vect = CountVectorizer(analyzer = clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis = 1) 

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [8]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators':[100, 150],
    'max_depth':[7,11,15],
    'learning_rate':[0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\dynamic_h\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\dynamic_h\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\dynamic_h\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\dynamic_h\Anaconda3\lib\site-packa

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,353.516538,3.132889,0.396316,0.043560,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.967713,0.981132,...,0.970361,0.005503,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,716.802505,101.432344,0.428775,0.163741,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965919,0.978437,...,0.970002,0.004338,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,544.022095,6.582808,0.432979,0.068480,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965022,0.979335,...,0.969643,0.005178,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,246.286867,12.384548,0.392709,0.024198,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965022,0.979335,...,0.968385,0.005742,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,378.450409,2.540026,0.373989,0.050960,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964126,0.977538,...,0.968385,0.004923,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [11]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\dynamic_h\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\dynamic_h\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\dynamic_h\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\dynamic_h\Anaconda3\lib\site-packa

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,584.067190,4.838024,0.458561,0.044244,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965022,0.978437,...,0.969643,0.004789,1,1.000000,1.0,1.000000,1.0,1.000000,1.000000,0.00000
5,11104.865202,8653.066880,0.307089,0.062728,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963229,0.978437,...,0.969643,0.005593,1,1.000000,1.0,1.000000,1.0,1.000000,1.000000,0.00000
2,380.782241,25.809825,0.437204,0.057006,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964126,0.976640,...,0.968565,0.004604,3,1.000000,1.0,1.000000,1.0,1.000000,1.000000,0.00000
0,210.436461,1.730359,0.416583,0.086502,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963229,0.979335,...,0.968385,0.005794,4,0.999775,1.0,0.999775,1.0,0.999775,0.999865,0.00011
4,4025.652886,6975.556987,0.413123,0.124540,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963229,0.974843,...,0.968205,0.004026,5,1.000000,1.0,1.000000,1.0,1.000000,1.000000,0.00000
